<center><h2>Exercices pratiques de webscraping</h2></center>
<br/>

In [1]:
# Installation des librairies tierces utilisées.
import pip
for pkg in ['requests', 'bs4']:
    pip.main(['install', pkg])

### Exercice 1 - Suivi de la "Une" du site Le Monde

L'objectif est ici de récupérer la liste des articles en "Une" du site http://www.lemonde.fr/ de manière automatisée.

_Note : le fichier `robots.txt` du site Le Monde interdit la navigation automatisée, aussi veillera-t-on à limiter
 au maximum le nombre de requêtes effectuées, dans cet exercice comme dans les suivants. Pour ce faire, mieux vaut
 séparer les cellules vouées au téléchargement des pages à analyser de celles où l'extraction d'information à lieu,
 puisque cette dernière est par nature soumise à un processus d'essais/erreurs, sans qu'il ne soit nécessaire de
 télécharger la page à chaque itération._


####  1.0 Récupérer la page d'accueil du site

In [2]:
import requests

response = requests.get('http://www.lemonde.fr/')
if response.status_code != 200:
    print('Échec de la requête: statut %s.' % response.status_code)

#### 1.1 Récupérer l'article en "Une".

Le premier objectif est de récupérer l'article en "Une" du journal _Le Monde_. Il s'agit d'extraire son titre, la ou les phrases d'accroche associées et l'url de la page de l'article.

**Correction** :

* L'étude de la page permet de repérer que l'article en Une se trouve dans une balise `<article>` de classe spécifique `titre_une`. Ces deux critères permettent alors de récupérer cet article à l'aide de `BeautifulSoup`.

* Au sein de la balise extraite se trouvent des sous-balises structurant les informations ciblées : le titre est dans une balise `<h1>`, le texte dans une balise `<p>` et l'url (relative) dans la propriété `href` d'une balise `<a>`.

* Optionnellement, on peut nettoyer le texte extrait.

In [3]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(response.text, 'html.parser')
article_une = soup.find('article', {'class': 'titre_une'})

In [4]:
import re

def clean_text(string):
    """Fonction de nettoyage textuel sommaire."""
    # Remplace les tabluations, sauts de lignes, etc. par des espaces.
    string = re.sub('\n|\r|\t|\xa0', ' ', string)
    # Retire les ' .' (séparateurs utilisés dans `soup.get_text`) inappropriés.
    string = re.sub('^\.', '', string.replace(' .', ''))
    # Retire les espaces inappropriés.
    return re.sub('  +', ' ', string).strip(' ')
    

In [5]:
{
    'titre': clean_text(article_une.find('h1').text),
    'texte': clean_text(article_une.find('p').text),
    'url': 'https://www.lemonde.fr' + article_une.find('a')['href']
}

{'texte': 'Editorial. L’appel d’Emmanuel Macron, le 25 novembre, à faire de l’égalité hommes-femmes une « grande cause du quinquennat » est assorti de mesures concrètes, mais pas de nouveaux moyens.',
 'titre': 'L’antisexisme, grande cause petits moyens 8',
 'url': 'https://www.lemonde.fr/idees/article/2017/11/27/la-grande-cause-de-l-antisexisme_5220938_3232.html'}

#### 1.2 Récupérer les articles en tête de section

Outre la Une, certains articles sont mis en avant sur la page d'accueil du journal _Le Monde_ par l'inclusion d'un petit texte d'accroche. Le second objectif consiste à repérer ces articles et à extraire les mêmes informations que pour la Une (titre, url et descriptif).

Attention, certaines urls ne sont pas faciles à identifier, du fait d'une technologie particulière permettant leur chargement lors du rendu de la page. Dans un premier temps, chercher à ne récupérer les urls que lorsqu'elles sont "faciles" à identifier. Optionnellement, chercher une astuce pour les retrouver (une proposition se trouve dans le corrigé de ce notebook).

**Correction** :

* Les articles ciblés font, comme la Une, l'objet d'une structuration au sein d'une balise `<article>` de classe spécifique `img_tt_chapo`.


* Les informations que l'on souhaite extraire sont structurées selon des sous-balises similaires au cas de la _Une_, si ce n'est que le titre est dans une balise `<h2>` et non `<h1>`.


* Dans certains cas cependant, l'url n'est pas directement accessible, car elle est chargée dynamiquement par un script à partir de l'attribut `data-href` d'une balise `<span>`. L'astuce proposée est alors de chercher parmi l'ensemble des urls disponibles sur la page s'il en est une qui contient les premiers mots du titre de l'article. Cette solution s'appuie sur la structure particulière des urls des articles sur le site du _Monde_, et n'est ni généralisable en toutes circonstances, ni une solution optimale ; sans doute peut-on faire mieux.

In [6]:
# Version "simple" de la solution, avec urls manquantes.
[
    {
        'titre': clean_text(article.find('h2').text),
        'texte': clean_text(article.find('p').text),
        'url': (
            ('https://www.lemonde.fr' + article.find('a')['href'])
            if article.find('a') is not None else ''
        )
    }
    for article in soup.find_all('article', {'class': 'img_tt_chapo'})
]

[{'texte': 'Les Vingt-Huit, réunis en comité d’appel après l’échec de toutes les tentatives précédentes pour s’accorder, doivent décider, lundi après-midi, de réautoriser ou non le pesticide de Monsanto pour cinq ans.',
  'titre': 'Après deux ans de négociations, l’UE doit statuer sur le glyphosate 28',
  'url': 'https://www.lemonde.fr/planete/article/2017/11/27/glyphosate-l-europe-en-echec_5220871_3244.html'},
 {'texte': 'Le tennisman nordiste a surmonté son stress pour offrir le cinquième point aux Bleus. Mais refuse d’endosser le costume de sauveur de la génération Tsonga.',
  'titre': 'Coupe Davis : l’heure de Lucas Pouille avait sonné',
  'url': 'https://www.lemonde.fr/tennis/article/2017/11/27/coupe-davis-l-heure-de-lucas-pouille-avait-sonne_5220986_1616659.html'},
 {'texte': 'Une éruption majeure du mont Agung est redoutée par les autorités. Des dizaines de milliers d’habitants de l’île indonésienne ont dû être évacués. L’aéroport international a été fermé.',
  'titre': 'Ce qu’i

In [7]:
# Fonctions auxiliaires pour trouver certaines des urls manquantes.

def get_href_from_title(titre, soup):
    """Cherche un lien parmi ceux d'une page concordant avec un titre donné."""
    # Nettoyage du titre.
    titre = retirer_accents(titre.lower())
    titre = re.sub('^ | $', '', re.sub('  +', ' ', re.sub('[^a-z ]', '', titre)))
    # Sous-sélection et formattage des trois premiers termes du titre.
    title_part = '-'.join(titre.split(' ')[:3])
    # Itération sur les urls liés dans la page, jusqu'à en trouver une qui concorde.
    for anchor in soup.find_all('a'):
        if title_part in anchor.get('href', ''):
            return 'https://www.lemonde.fr' + anchor['href']

        
def retirer_accents(text):
    """Remplace les caractères accentués par leur équivalent sans accent."""
    charmap = [
            ('àâ', 'a'), ('ç', 'c'), ('éèêë', 'e'),
            ('îï', 'i'), ('ôö', 'o'), ('ùûü', 'u')
        ]
    for character in charmap:
        text = re.sub('[%s]' % character[0], character[1], text)
    return text

In [8]:
# Version "complexe" utilisant les fonctions précédentes.

def parse_head_article(article, soup):
    """Extrait le titre, le texte et l'url associés à un article "mis en avant"."""
    parsed = {
        'titre': clean_text(article.find('h2').text),
        'texte': clean_text(article.find('p').text)
    }
    parsed['url'] = (
        get_href_from_title(parsed['titre'], soup)
        if article.find('a') is None
        else 'https://www.lemonde.fr' + article.find('a')['href']
    )
    return parsed

articles = [
    parse_head_article(article, soup)
    for article in soup.find_all('article', {'class': 'img_tt_chapo'})
]

In [9]:
# Affichage "lisible" des résultats.

def format_article(article_dict):
    return '%s\n\n%s\n\n%s' % (article_dict['titre'], article_dict['texte'], article_dict['url'])


print('\n\n\n'.join(map(format_article, articles)))

Après deux ans de négociations, l’UE doit statuer sur le glyphosate 28

Les Vingt-Huit, réunis en comité d’appel après l’échec de toutes les tentatives précédentes pour s’accorder, doivent décider, lundi après-midi, de réautoriser ou non le pesticide de Monsanto pour cinq ans.

https://www.lemonde.fr/planete/article/2017/11/27/glyphosate-l-europe-en-echec_5220871_3244.html


Coupe Davis : l’heure de Lucas Pouille avait sonné

Le tennisman nordiste a surmonté son stress pour offrir le cinquième point aux Bleus. Mais refuse d’endosser le costume de sauveur de la génération Tsonga.

https://www.lemonde.fr/tennis/article/2017/11/27/coupe-davis-l-heure-de-lucas-pouille-avait-sonne_5220986_1616659.html


Ce qu’il faut retenir de l’éruption du volcan Agung à Bali

Une éruption majeure du mont Agung est redoutée par les autorités. Des dizaines de milliers d’habitants de l’île indonésienne ont dû être évacués. L’aéroport international a été fermé.

https://www.lemonde.fr/planete/article/2017/11

#### 1.3 Récupérer les titres et adresses de tous les articles listés sur la page d'accueil

La page d'accueil du journal _Le Monde_ comporte des liens vers de nombreux articles, différemment mis en avant mais se composant _a minima_ d'un titre et d'un lien pointant vers l'url de la page de l'article. Le dernier objectif consiste à repérer ces articles dans le html récupéré, et à extraire pour chacun son titre et son url.

**Correction** :

* Tous les articles sont rangés dans la section "articles" du site, une solution simple est donc d'extraire l'ensemble des urls présents dans le html (attributs `href` des balises `<a>`), et de filtrer celles répondant à ce schéma.


In [10]:
def parse_lien(tag):
    """Parse un lien d'article sur le site du Monde."""
    url = tag['href']
    if tag.text != '':
        text = tag.text
    elif tag.find('h1') is not None:
        text = tag.find('h1').text
    elif tag.find('h2') is not None:
        text = tag.find('h2').text
    else:
        text = ''
    return {re.sub('^\d\d:\d\d', '', clean_text(text)): url}

In [11]:
liens = [parse_lien(a) for a in soup.find_all('a') if '/article/' in a['href']]

print('\n\n'.join(map(str, liens)))

{'L’antisexisme, grande cause petits moyens 8': '/idees/article/2017/11/27/la-grande-cause-de-l-antisexisme_5220938_3232.html'}

{'Les principales annonces de Macron pour l’égalité entre les femmes et les hommes 171': '/politique/article/2017/11/25/l-egalite-entre-les-femmes-et-les-hommes-proclamee-grande-cause-du-quinquennat_5220375_823448.html'}

{'Violences faites aux femmes : les mesures de Macron « irréalisables » sans moyens regrettent les associations féministes 28': '/societe/article/2017/11/25/violences-faites-aux-femmes-des-mesures-irrealisables-sans-moyen-regrettent-les-associations-feministes_5220442_3224.html'}

{'Harcèlement sexuel : l’inaction des pouvoirs publics mise en cause 58': '/societe/article/2017/11/07/l-inaction-des-pouvoirs-publics-contre-le-harcelement-sexuel-mise-en-cause_5211125_3224.html'}

{'Violences faites aux femmes : les féminicides, ces meurtres encore invisibles 242': '/societe/article/2017/11/24/les-feminicides-des-meurtres-invisibles_5219790_3224.